## This is a script to stack VANDELS spectra

The following steps are carried out to perform the stacking:
- Point to a folder containing fits files and read in the spectra
- Convert the spectra to rest frame using the redshift value given in the header file
- Create a rest-frame wavelength grid assuming a median wavelength on to which individual spectra would be resampled
- Perform a weighted average of the spectra, where the weights are based on the errors on the individual spectra
- Propagate the errors and save the stacked spectrum along with the errors!


Python packages required for essential functions:
- numpy
- astropy
- mpdaf (pip/pip3 install mpdaf)
- matplotlib (for plotting)

Additionally, to make life easier it is also useful to place the mpvandels.py file in the same folder as this notebook. This custom python package helps read in the spectra and easily convert them to rest-frame

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.table import Table

from astropy.stats import sigma_clipped_stats

import os
import glob
import matplotlib.pyplot as plt

import mpdaf as mp
import astropy.units as u

import mpvandels as mpv
from mpdaf.obj import Spectrum, WaveCoord

import math

### Define restframe wavelengths and then redshift them for fitting
### Selected lines only

rest_lines = np.array([1215.24, 1240.81, 1262., 1302., 1335.31, 1402.8, 1549.6, 1640.4, 1666., 1892., 1908.], dtype=float)
linewidths = np.array([50, 20, 25, 10, 15, 5, 15, 20], dtype=float)

line_names = ['Lya', 'NV', 'SiII', 'OI', 'CII', 'SiIV', 'CIV', 'HeII', 'OIII]','SiIII]', 'CIII]']

In [ ]:
### Define source director with all the spectra
SPEC_DIR = "\Users\aayushsaxena\VANDELS\spectra\""
os.chdir(SPEC_DIR)


### Define arrays to append individual spectra and properties
spectra = []
errors = []
weights = []

### Flux at 1500A used to normalise
FUV = []
### Redshift
ztar = []

### Keep track of number of sources
sourcecounter = 0

speclist = glob.glob("*.fits")
for spec in speclist:
    specdata, z = mpv.readvandels(spec) ### Read in the observed frame spectrum and redshift
    restspec = mpv.restframespec(spec, z) ### convert to rest frame
    
    ### append z info to be used later for resampling
    ztar.append(z)
    
    ### Normalisation
    fUV = restspec.mean(lmin=1480, lmax=1520, weight=True, unit=u.Angstrom)
    ### Measure variance in the region around 1500 A to determine error on the spectrum
    specvar = np.mean(restspec.subspec(lmin=1480, lmax=1520, unit=u.Angstrom).var)
    specvar = specvar/fUV[0] * 100.
    
    ### Normalise the spectrum using flux at ~1500 A
    restspec.data = restspec.data/fUV[0]
    restspec.var = restspec.var/fUV[0]
    
    ### append spectrum and errors to a list
    spectra.append(restspec)
    errors.append(restspec.var)
    
    ### Append UV flux and weights
    FUV.append(fUV[0])
    weights.append(specvar)
    
    sourcecounter+=1
    
print("%d sources read in!" %sourcecounter)


### choose the cdelta value of the new wavelength grid based on the median redshift of the sample to be stacked
### calculate median z from all the redshifts of individual sources
zmed = np.median(ztar)
print("Median z = ", zmed)

### Define the cdelta value (using the CDELTA in the observed frame)
b=2.55299997329712/(1+zmed) # cdelt normalised to the mean redshift

print("CDELT = ", b, "for new wavelength array defined!")

In [ ]:
### Resampling and stacking the spectra

### Define empty arrays to append fluxes and weights, calculated from the error spectrum
stackfluxes = []
stackerrors = []
stackweights = []

for i in range(len(spectra)):
    ### Resample the Spectrum object to the new wavelength grid
    specs = spectra[i]
    errs = errors[i]
    
    weight = weights[i]
    
    ### resample the spectrum using the wavelength step of the median redshift, starting from 1200 A
    specs = specs.resample(step=b, start=1200, shape=1387, unit=u.Angstrom)
    
    ### Replace NaNs with zeros in case there is no wavelength coverage for certain sources
#     where_are_NaNs = np.isnan(specs.data)
#     specs.data[where_are_NaNs] = 0.
    
#     specs.var[where_are_NaNs] = 0.

    ### Mask the array values that have NaNs as a result of incomplete wavelength coverage
    specs.data = np.ma.masked_array(specs.data, np.isnan(specs.data))
    specs.var = np.ma.masked_array(specs.var, np.isnan(specs.data))
    
    ### Mask the array values that have bad sky subtraction
    ### Iterate over spectrum pixels and mask 20 sigma outliers
    try:
        mean, med, std = sigma_clipped_stats(specs.data, sigma=3)
    
        spec_mask = np.zeros(len(specs.data))
        for j in range(len(specs.data)):
            if abs(specs.data[j]) > med + 20*std:
                specs[j] = 1

        specs.data = np.ma.masked_array(specs.data, spec_mask)
        specs.var = np.ma.masked_array(specs.var, spec_mask)
        
    except ValueError:
        print(specs, "skylines could not be masked")
            
    ### Append the fluxes and weights. Weights = 1/(err^2)
    stackfluxes.append(specs.data)
    stackerrors.append(specs.var)
    
    stackweights.append(1/(weight**2))
        
        
### replace NaNs in the weights
for i in range(len(stackweights)):
    if math.isnan(stackweights[i]):
        stackweights[i] = 0.

        
### Stack. This is done by weighted averaging of the fluxes. The weights are stored in the stackweights array
medflux = np.ma.average(stackfluxes, axis=0, weights=stackweights)

# medvar = np.ma.median(stackweights, axis=0)
### Stack the error spectra
med_error = np.ma.std(stackfluxes, axis=0)

### print the median FUV
med_FUV = np.nanmedian(FUV)

### Define the stacked spectrum using the new wavelength grid. Also record the error spectrum
stackedwave = WaveCoord(crval=1200, cdelt=b, cunit=u.Angstrom)
stackedspec = Spectrum(wave=stackedwave, data=medflux, var=med_error)

# allwave = WaveCoord(crval=1200, cdelt=b, cunit=u.Angstrom)
# allspec = Spectrum(wave=stackedwave, data=medflux, var=med_error)

print("Stacking complete!")


In [ ]:
### If happy with the results, save the stacked spectrum
### The spectrum is saved as a MPDAF object, and can be easily read in using MPDAF
### The spectrum is also saved as a normal 1D fits file with error information, so can in principle be read
### using tools other than MPDAF as well.

stackedspec.write("/Users/aayushsaxena/Desktop/VANDELS/spectrum-stacked.fits")